# Hybrid

### Copy data from Drive

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

# !cp "/gdrive/My Drive/recsys/src.zip" /content
# !cp "/gdrive/My Drive/recsys/data.zip" /content
# !cp "/gdrive/My Drive/recsys/utilities.py" /content
# !cp "/gdrive/My Drive/recsys/tuner" /content


# !unzip src.zip
# !unzip data.zip

### Import libraries

In [128]:
import pandas as pd
import numpy as np
from scipy import sparse as sps
import os
import csv

from utilities import load_sparse_matrix

### Load dataset

In [129]:
ICM = load_sparse_matrix("icm.npz")
URM = load_sparse_matrix("urm.npz")
URM_test = load_sparse_matrix("urm_test.npz")
URM_train = load_sparse_matrix("urm_train.npz")
URM_validation = load_sparse_matrix("urm_val.npz")
URM_train_validation = load_sparse_matrix("urm_train_val.npz")

Loaded icm.npz
Loaded urm.npz
Loaded urm_test.npz
Loaded urm_train.npz
Loaded urm_val.npz
Loaded urm_train_val.npz


### Parameters

In [130]:
output_folder_path = "tuner/"
n_cases = 100
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

### Load recommenders

In [131]:
recommenders = []

In [132]:
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

rec = ItemKNNCFRecommender(URM)
rec.load_model(
    output_folder_path, 
    file_name=rec.RECOMMENDER_NAME + "_best_model_last.zip"
)
recommenders.append(rec)

ItemKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'tuner/ItemKNNCFRecommender_best_model_last.zip'
ItemKNNCFRecommender: Loading complete


In [133]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

rec = P3alphaRecommender(URM)
rec.load_model(
    output_folder_path, 
    file_name=rec.RECOMMENDER_NAME + "_best_model_last.zip"
)
recommenders.append(rec)

P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
P3alphaRecommender: Loading model from file 'tuner/P3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete


### Custom hybrid model

In [134]:
from src.Recommenders.BaseRecommender import BaseRecommender


class CustomHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    
    Hybrid of multiple recommenders weighted on the given alphas.
    """

    RECOMMENDER_NAME = "CustomHybridRecommender"

    def __init__(self, URM_train, recommenders):
        super(CustomHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommenders = recommenders

    def fit(self, **kwargs):
        # Read multiple alphas
        alphas = []
        for k, v in kwargs.items():
            alphas.append(v)

        # Number of alphas should be equal to number of recommenders
        assert len(alphas) == len(self.recommenders)

        # Normalize alphas sum to 1
        # Sum all alphas values
        sum = 0
        for alpha in alphas:
            sum += alpha

        # Calculate normalized alphas
        for i in range(len(alphas)):
            alphas[i] += alphas[i] / sum

        self.alphas = alphas

    def _compute_item_score(self, user_id_array, items_to_compute):
        rec = self.recommenders[0]
        alpha = self.alphas[0]
        item_weights = alpha * rec._compute_item_score(user_id_array)

        for i in range(1, len(self.recommenders)):
            rec = self.recommenders[i]
            alpha = self.alphas[i]
            item_weights += alpha * rec._compute_item_score(user_id_array)

        return item_weights


### Tuning

In [135]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "alpha1": Real(0, 1),
    "alpha2": Real(0, 1),
}

In [136]:
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, recommenders],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation, recommenders],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [137]:
from src.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 759 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 322 ( 0.8%) Users that have less than 1 test interactions


In [138]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = CustomHybridRecommender

hyperparameterSearch = SearchBayesianSkopt(
    recommender_class,
    evaluator_validation=evaluator_validation,
    evaluator_test=evaluator_test
)

In [139]:
hyperparameterSearch.search(
       recommender_input_args,
       recommender_input_args_last_test = recommender_input_args_last_test,
       hyperparameter_search_space = hyperparameters_range_dictionary,
       n_cases = n_cases,
       n_random_starts = n_random_starts,
       save_model = "last",
       output_folder_path = output_folder_path, # Where to save the results
       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
)

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha1': 0.5964026484014824, 'alpha2': 0.7970223308985851}
CustomHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
EvaluatorHoldout: Processed 40870 (100.0%) in 1.98 min. Users per second: 343
SearchBayesianSkopt: New best config found. Config 0: {'alpha1': 0.5964026484014824, 'alpha2': 0.7970223308985851} - results: PRECISION: 0.0861023, PRECISION_RECALL_MIN_DEN: 0.1947186, RECALL: 0.1885394, MAP: 0.0480564, MAP_MIN_DEN: 0.1145472, MRR: 0.2304163, NDCG: 0.1668815, F1: 0.1182171, HIT_RATE: 0.4685588, ARHR_ALL_HITS: 0.3292959, NOVELTY: 0.0036647, AVERAGE_POPULARITY: 0.2514268, DIVERSITY_MEAN_INTER_LIST: 0.8781306, DIVERSITY_HERFINDAHL: 0.9878109, COVERAGE_ITEM: 0.5435855, COVERAGE_ITEM_HIT: 0.4772597, ITEMS_IN_GT: 0.8718535, COVERAGE_USER: 0.9817675, COVERAGE_USER_HIT: 0.4600159, USERS_IN_GT: 0.9817675, DIVERSITY_GINI: 0.0328570, SHANNON_ENTROPY: 8.2445473, RATIO_D

KeyboardInterrupt: 

### Save recommendations

In [140]:
from utilities import save_recommendations

hybrid_recommender = ItemKNNCFRecommender(URM)
hybrid_recommender.load_model(
    output_folder_path, 
    file_name=rec.RECOMMENDER_NAME + "_best_model_last.zip"
)
save_recommendations(hybrid_recommender)

ItemKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'tuner/P3alphaRecommender_best_model_last.zip'
ItemKNNCFRecommender: Loading complete
Saving recommendations finished!                                                                    
